# Creating an American Football Reinforcement Learning Enviornment 

In [1]:
# Package dependencies
import pygame
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import nfl_data_py as nfl

pygame 2.5.2 (SDL 2.28.3, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Custom dependencies
from FootballPlay import FootballPlay
from PreprocessTrackingData import PreprocessTrackingData

In [3]:
# Turn off pandas column limits
pd.set_option('display.max_columns', None)

In [4]:
# Read in the data
tracking = pd.read_csv("data/tracking_week_1.csv")
plays = pd.read_csv("data/plays.csv")
players = pd.read_csv("data/players.csv")
colors = pd.read_csv("data/colors.csv")

In [5]:
# Select game and play
gameId = 2022090800
playId = 56
nflId = 41239

In [6]:
# Define preprocessor
preprocessor = PreprocessTrackingData(tracking, 
                                      plays,
                                      players,
                                      colors,
                                      gameId, 
                                      playId
                                      )

In [7]:
# Get processed data
play_data = preprocessor.get_processed_data()

# Find the maximum frame ID
max_frames = play_data['frameId'].max()

In [8]:
# Define the environment
env = FootballPlay(play_data, nflId, render_mode='human', max_frames=max_frames)

In [9]:
for _ in range(1000):
    observation, info = env.reset()
    done = False

    while not done:
        action = env.action_space.sample()  # Sample an action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated:
            done = True  # Mark the episode as done

env.close()

KeyboardInterrupt: 